# Section 5: Models

In this section we present the most used machine learning models in problems involving security, including their theory, algorithms and codes of how to use them. Among the models, there are classifiers, detectors and clustering techniques, all of them explained here.

## Classifiers

Classifiers aim to classify a given input sample into a previously known class by them during the training. The training is the step that the classifier learns the patterns of each class with the data presented to it (together with their labels), adapting its parameters to the problem. This type of problem is known as supervised learning [Bishop 2006]. After the training, the model can be used to classify any unknown data, allowing it to be effectively used. Here we present the following classifiers: K-Nearest Neighbors, based in neighborhood, Random Forest, an ensemble based in decision trees, Support Vector Machines, based in the construction of a optimal hyperplane, and Multi-Layer Perceptron, a neural network type used a lot in deep learning.

### K-Nearest Neighbors (KNN)

K-Nearest Neighbors (KNN) is a machine learning model based in distance, which classification of a new instance is based on the distance of the $k$ training samples most closer to a given testing sample. Thus, a new unknown sample will be classified as being from the most prevalent classes between these $k$ samples, as shown in figure below [Michie et al. 1994], where a new instance will be classified as red when $k=3$, green, when $k=5$ and unknown when $k=6$ (the result is a draw, that is why even numbers are not recommended in binary classification - when there are just two classes).

<img src="imgs/knn1.png" align="center">

Generally, the distance used by KNN is the Euclidean distance. Given an instance $x$, described by $(a_{1}(x),a_{2}(x),...,a_{n}(x))$, where $a_{i}(x)$ is the $i$-th feature, the distance between two instances $x_{i}$ and $x_{j}$ is defined by the equation $ d(x_{i},x_{j}) = \sqrt{ \sum \limits_{r=1}^{n} (a_{r}(x_{i})-a_{r}(x_{j}))^{2}) } $.

### Random Forest

Random forest is a classifier that consists of a collection (ensemble) of tree-structured classifiers, each of them trained on bagged data using random selection of features (bootstrap aggregating or bagging) and cast an vote for the most popular class for a given input [Breiman 2001]. The figure below shows an example of a random forest, containing $L$ trees using bagging. To better understand this ensemble. we will explain how a decision tree works. 

<img src="imgs/randomforest1.png" width="500" align="center">

A decision tree is basically a classifier that creates a set of if-then rules to classify new samples. These rules improve human readability, since it makes easy to understand the model and which attributes are important. The classification of new instances is done by sorting them down the tree from the root to some leaf node, which provides the classification of the instance. Each node of the tree represents a test of some attribute of the instance, and each branch descending from that node corresponds to one of the possible values for this attribute [Mitchell 1997]. Figure below shows an example of a decision tree used to classify if a given moment of a day is good do play tennis (based on weather conditions).

<img src="imgs/randomforest2.png" width="500" align="center">

The training of a decision tree is based in two measures: entropy and information gain. The entropy, represented by the equation $Entropy(S) = \sum_{i=1}^{c} - p_{i}\log_{2}(p_{i})$, where $S$ is a set of training samples and $c$, the number of classes, measures the homogeneity of the samples. 

The information gain, as shown in equation $Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_{v}|}{|S|} Entropy(S_{v})$, is used as an attribute selection measure. The tree is built according to the information gain of the attribute $A$, since you always pick the attribute with higher information gain as splitting attribute, to create new branches for each value of this attribute (or intervals, in the case of numerical features) [Mitchell 1997].

The algorithm for training a decision tree (the basic algorithm, called ID3), given a set of *examples* (training samples), the *target attribute* (label, the attribute to be predicted
by the tree) and the *attributes* (list of the attributes of the samples, excluding the label), is the following [Mitchell 1997]:

1. Create a root node for the tree.
2. If all *examples* are positive, return the single-node tree root, with $label=+$.
3. If all *examples* are negative, return the single-node tree root, with $label=-$.
4. If *attributes* is empty, return the single-node tree root, with $label=$ most common value of *target attributes* in *examples*.
5. Otherwise, do the following:
   1. $A=$ the attribute from *attributes* that best  classifies *examples*. The best attribute is the one with highest information gain.
   2. The decision attribute for root is $A$.
   3. For each possible value $v_{i}$ of $A$:
      1. Add a new tree branch below root, corresponding to the test $A=v_{i}$.
      2. Let $examples_{v_{i}}$ be the subset of  *examples* that have value $v_{i}$ for $A$:
         1. If $examples_{v_{i}}$ is empty, them, below this new branch, add a leaf node with $label=$ most common value of *target attributes* in *examples*.
         2. Else, below this new branch, create a new subtree, going back to step 1 using a subset of *examples* ($examples_{v_{i}}$) and *attributes* ($attributes-\{A\}$).

### Support Vector Machine (SVM)

The Support Vector Machine (SVM), originaly developed for binary classification, finds, in linearly separable problems, the construction of a hyperplane as a decision surface (a boundary), such that the separation between the samples is maximal. When the patterns are non-linearly separable, the SVM finds a mapping function, which projects the data in a space where the data is linearly separable. The main idea of this classifier is to maximize the hyperplane margin from the training data. An optimal hyperplane is the one whose margin distance to the positive class is the same margin distance to the negative class. The figure below illustrates an optimal hyperplane defined by the support vectors, the training samples  most closer to it [Cortes and Vapnik 1995, Fradkin and Muchnik 2006].

<img src="imgs/svm1.png" width="500" align="center">

In most of times, the problems are not linearly separable. Due to that, it's necessary to project the data in a space where they are linearly separable, called feature space. The kernel function is responsible for this projection and this process is called kernel trick. After projected, it's possible to find a hyperplane that separates the data in that space. The figure below exemplifies the use of the kernel trick to project the data in another dimension.

<img src="imgs/svm2.png" width="800" align="center">

The decision function responsible for building the hyperplane is defined by equation $f(x) = \sum\limits_{i}\alpha_{i}y_{i}K(x,x_{i})+b$, where $K$ is 
the kernel function, $\alpha$ and $b$ are parameters found during the training, $x_{i}$ are the feature vectors and $y_{i}$, their labels.

Since the majority of the real problems involve more than two classes and the SVM is a binary classifier, it's necessary to use a different decision approach. The most common is the one
versus all, also called one versus the rest [Manning et al. 2008b]. In this approach, there are $q$ classifiers (SVMs), where $q$ is the number of classes. Each SVM $c_{i}$ is trained to the class $i$, using as counterexample the samples from another classes. The final decision can be made through a "vote counting" [Milgram et al. 2006].

### Multi-Layer Perceptron (MLP)

A neural network is a computational model of a human brain that can be composed of hundreds or thousands of neurons (processing units). In general, it's a machine that is designed to model the way in which the brain performs a particular task. The smallest component of a neural network is a perceptron, a processing unit that simulates a neuron [Haykin 2009]. Figure below shows the structure of a perceptron, where $x=\{x_{1}, x_{2}, ..., x_{n}\}$ are the signals (input), $w=\{w_{1}, w_{2}, ..., w_{n}\}$ are the weights, $\varphi(.)$ is the activation function and $y$ is the output. 

<img src="imgs/mlp1.png" width="400" align="center">

The equation $y = \varphi (\sum_{i=1}^{n} w_{i} \times x_{i} + b)$ presents the output ($y$) of a perceptron. The input signals are multiplied to the weights, that are adapted on an iteration-by-iteration basis, using an error-correction rule known as the perceptron convergence algorithm. It also includes an bias ($b$), which has the effect of increasing or lowering the net input of the activation function. The activation function is responsible for determining the shape and intensity of the values transmitted from one neuron to another (or to the output), limiting the amplitude of the output [Haykin 2009]. 

The training of a single perceptron helps to understand better the operation of a neural network. The algorithm is the following [Haykin 2009]:

1. Initialize weights and bias with small random values.
2. Apply current sample input pattern and check the network output ($y$).
3. Calculate output error ($e$), comparing it (the output $y$) to the expected value ($t_{j}$), as shown in equation $e = t_{j} - y$.
4. If the output error is equal zero ($e=0$), it means that the output is correct. In that case, a new sample is presented, going back to the step 2. 
5. Otherwise, if the output error is different from zero ($e \neq 0$), it's necessary to update the weights and the bias, as shown in equations $w_{j}=w_{j}' + e \times x_{j}$ and $b = b' + e$, respectively.
6. Go back to step 2 and present a new sample to the network. The stopping criteria can be based in iterations number, average error rate, accuracy, etc.

Despite of being efficient, a single perceptron can only solve linearly separable problems, which, in most of times, are not present in the real world, i.e., the presence of non-linear 
problems is very common in real problems. Due to that, neural networks usually combine more than one neuron, which makes it possible to them separate non-linear problems [Haykin 2009]. Figure below shows two examples of problems, the first one (left) linear and the second (right), non-linear. As the decision boundary of the perceptron is defined by a line, it solves problems like the first one. A more complex network, such as multilayer perceptron, composed of multiple neurons, can solve the second one.

<img src="imgs/mlp2.png" width="800" align="center">

A multilayer perceptron (MLP) neural network is composed by source nodes, that are the input, one layer or more of perceptrons, called hidden layers, and the output neurons. All the layers, except the input, are composed by neurons, each one of them initialized with random weights and bias. This type of network is progressive, i.e., the neurons of a certain layer
are just connected to the next layer. Thus, the input passes through all existing layers. The number of input nodes is the dimensionality of the input data and the number of neurons 
in the output is generally composed by the number of classes of the problem (in this case, each neuron represents one class, i.e., the output value of the neuron is directly related
to its respective class. The higher the value, the higher the chances of that sample being of that class) [Haykin 2009]. The figure below presents a multilayer perceptron neural network with two hidden layers and three output neurons.

<img src="imgs/mlp3.png" width="800" align="center">

An important algorithm for neural networks in general is back propagation, a computationally efficient method for the training of multilayer perceptrons. Briefly, this algorithm implements gradient descent in weight space for a 
multilayer perceptron, efficiently computing partial derivatives of an approximating function $F(w,x)$ realized by the network, adjusting its weights according to the input [Haykin 2009].

## Detectors

The detection of change is a fundamental theme for a lot of problems related to security, given that the distribution of the data is in constant change to evade security systems. These changes are called concept drift, the situation in which the relation between the input data and the target variable (variable that needs to be learned, which is often the class variable), changes over time. It generally happens when there is a change in a hidden context, which makes it difficult to handle, since this problem spans across different research 
fields. Each example in the input data is represented by a feature vector $x=[x_1,x_2,...,x_L]$, where $L$ is the number of features that are used to determine it's class $y$, according to the *a posteriori* probabilities $P(y,x)$. $P(x)$ is defined as features distribution and $P(y)$ as prior probabilities. In the literature, there are two types of concept drift, both described below [Wang et al. 2011, Gama et al. 2014].

The virtual concept drift happens when the distribution of the incoming data changes, i.e., $p(x)$ changes, without changing $p(y,x)$. Figure below shows an example of virtual drift with changes in the data distribution at times $t$ and $t+1$, where the red class is more prevalent but it does not lead to changes in the best boundary between them [Almeida et al. 2015].

<img src="imgs/drift1.png" width="800" align="center">

To illustrate the problem caused by the virtual concept drift, figure below presents the distribution of a feature for the blue and red classes (presented in previous figure) at both 
times $t$ and $t+1$. The vertical dashed line in both figures is the threshold that separates both classes (the misclassification cost of the red class is higher than the blue one), that are equally distributed in $t$, different from $t+1$, which has greater presence of the red class. Despite of the fact that the mean and standard deviation of both classes remains unchanged, with the threshold in the same position, i.e., keeping the same classifier unchanged, the probability of finding a red object as a blue one is increased, as shown in the 
dark red area [Almeida et al. 2015]. 

<img src="imgs/drift2.png" width="800" align="center">

The real concept drift happens when there is a change in $P(y,x)$ with or without changing $p(x)$, i.e., the relation between the classes and the feature vectors changes over time. A classic example is related to the e-mail spam problem, where an e-mail represented by a feature vector $x_{e}$ can be considered as a spam at a given time $t$ and cannot be at $t+1$, due to user behavior changes. As an example of real concept drift, figure below shows a two class problem with a *a posteriori* probabilities drift, causing changes in the boundaries at time $t$ and $t+1$ and forcing an update in the classifier [Almeida et al. 2015].

<img src="imgs/drift3.png" width="800" align="center">

For both types of concept drift, there are detectors, each one using a different strategy to analyze the changes that occur as time goes by. In this course, we selected three detectors that are commonly used in the literature: DDM (Drift Detection Method [Gama et al. 2004]), EDDM (Early Drift Detection Method [Baena-Garcıa et al. 2006]) and ADWIN (ADaptive WINdowing [Bifet and Gavaldà 2007]).

### Drift Detection Method (DDM)

The general idea of Drift Detection Method (DDM) is to monitor the error rate of a model while the samples are presented to the classifier in sequence, as a data stream [Gama et al. 2014]. Usually, when the data distribution is stationary, the error should drop or keep stable as more data is used. When the error rate increases, DDM uses it as evidence to detect a concept change [Albert Bifet 2018]. Given $p_{t}$ the error rate of the classifier and $s_{t}=\sqrt{\frac{{}p_{t}(1-p_{t})}{t}}$ the standard deviation, both at time $t$, DDM saves the lowest error rate $p_{min}$ and lowest standard deviation $s_{min}$ observed until time $t$ and then performs the following checks [Albert Bifet 2018]:

* If $p_{t}+s{t} \geq p_{min}+2*s_{min}$, it is considered that the model is in a warning stage, suggesting that a drift is starting. After this point, all the samples are stored in a buffer, as a change may be about to happen.
* If $p_{t}+s{t} \geq p_{min}+3*s_{min}$, it is considered that a change happened. The model used is discarded and a new one is built using the samples stored in the buffer since the warning happened. The values of $p_{min}$ and $s_{min}$ are reseted.

Although generic and simple to use, DDM may be too slow to respond to changes in some cases, as many samples can be observed until the drift level is effectively activated, and can even store too many samples in memory in this warning-drift interval [Albert Bifet 2018]. 

### Early Drift Detection Method (EDDM)

Early Drift Detection Method (EDDM) works similar to DDM. However, instead of considering the error rate, EDDM considers the distance between two errors. According to the authors, as a model is being trained (with new samples), it improves its predictions and the distance between two errors increases. Given $p_{i}$ the average distance between two errors and $s_{i}$ its standard deviation, EDDM stores both values when $p_{i}+2*s_{i}$ reaches its highest values (getting $p_{max}$ and $s_{max}$), i.e., $p_{max}+2*s_{max}$ is the point that the error distance distribution is maximum [Baena-Garcıa et al. 2006]. As the model is updated and tested with new samples, EDDM performs the following checks, given two parameters $\alpha$ and $\beta$:

* If $\frac{p_{i}+2*s_{i}}{p_{max}+2*s_{max}} < \alpha $, it is considered that the model is in a warning stage, storing the samples seen after this point in a buffer, suggesting the beginning of a possible change.
* If $\frac{p_{i}+2*s_{i}}{p_{max}+2*s_{max}} < \beta $, it is considered that a change happened, discarding the model used so far and building a new one using the data stored in the buffer since the warning happened. The $p_{max}$ and $s_{max}$ are reseted.

In the paper that EDDM was proposed [Baena-Garcıa et al. 2006], EDDM was better in all the experiments when compared to DDM, even in datasets with a great amount of noise. However, the problem related to memory usage is the same as before, once that when entering in a warning stage, the model stores all the samples until a drift happen.

### ADaptive WINdowing (ADWIN)

The ADaptive WINdowing (ADWIN) detector acts different from DDM and EDDM, keeping statistics from the variable size sliding windows, that are used to compute the average of the change observed "cutting" these windows in different points. The property of these windows must be statically consistent with the hypothesis "there were no changes in the average value inside the window", i.e., an old part of the window is discarded only if there was enough evidence that the average value differs from the rest of the window [Bifet and Gavaldà 2007]. The ADWIN has as parameter a confident value $\delta \in (0,1)$ and a test $T(W_{0}, W_{1}, \delta)$ that compare the average of two windows $W_{0}$ and $W_{1}$, that decides if both belong to the same distribution, based on the following checks [Albert Bifet 2018]:

* If $W_{0}$ e $W_{1}$ belong to the same distribution, then, with probability of at least $1-\delta$, the test answers that there was no change.
* If $W_{0}$ e $W_{1}$ are from two different distributions whose average differs by a large amount than $\epsilon(W_{0}, W_{1}, \delta)$, so, with a probability of at least $1-\delta$, the test answers that there was no change.

Different from DDM and EDDM, ADWIN may be more efficient in relation to the memory usage, given that it is not necessary to create a buffer to save samples, we just need to use the size of the "cropped" windows when detecting a change.

## Clustering

Sometimes the data are available without a label, which makes it difficult to use conventional classifiers. To this type of problem, it is used non-supervised learning [Bishop et al 2006], usually using clustering algorithms, which objective is to group similar patterns in groups that can or cannot be known, depending of the problem domain. In this course, we are going to present two clustering algorithms: K-Means and DBScan.

### K-Means

K-Means consists in grouping a set of data in $k$ groups, that are defined based on the average of their values, i.e., a certain sample belongs to the group that is closer to it [Michie et al. 1994]. The number of groups $k$ is defined by the user and corresponds to the number of classes generated after running the algorithm. Given the number of groups $k$, K-Means algorithm works in the following way [Rogers and Girolami 2011]:

1. Choose a number $k$ of groups. For each group, a random value is chosen at the beginning, considering it as being the centroid of the group.
2. For each dataset sample, its group is considered as being the one that is closer to it.
3. After labeling all data, a new centroid is computed for each group (through the average of each sample).
4. If a criteria is not met (iterations number, average variation of each centroid, etc), go back to step 2 until the criteria is reached.
5. At the end, $k$ different groups will be created for the dataset.

### Density-Based Spatial Clustering of Applications with Noise (DBScan)

Density-Based Spatial Clustering of Applications with Noise} (DBSCAN) is an algorithm that clusters points that are near others based on a distance measure (usually Euclidean distance) and a number of minimum points, marking as outliers points that are in low-density areas. The DBSCAN is composed by basically two parameters: *eps* ($\epsilon$), which specifies how close points must be to be considered part of a group, i.e., if the distance between two points is lower or equal this value, these points are considered neighbors (of the same group); and *minPoints*, the minimum number of points to create a dense area, i.e., it is necessary at least *minPoints* for an area to be considered dense. In general, values lower than *eps* ($\epsilon$) are preferable to many datasets e the bigger the dataset, bigger the *minPoints* value recommended to be used [Ester et al. 1996].

The DBSCAN algorithm works in the following way [Schubert et al. 2017]:

1. Find the points that are in a distance *eps* ($\epsilon$) of each point and find the central points with more than *minPoints* neighbors.
2. Find all the connected components to the central points in the neighborhood graph, ignoring all central points.
3. Assign each non-central point to the nearest cluster if it is at a distance *eps* ($\epsilon$), otherwise consider it as being a noise.

One of the advantages of DBSCAN in relation to K-Means is that it does not need to specify a number of clusters and it can find groups from many different ways [Ester et al. 1996].

## Libraries

In this course we present three libraries that the presented algorithms can be used: Scikit-Learn [Pedregosa et al. 2011], Scikit-Multiflow [Montiel et al. 2018] and Keras [Chollet et al. 2015]. All of them are open source, have an active community and are complementary in the majority of their functions. In this subsection, we are going to focus in how to use the classifiers of each mentioned library (without evaluation, which is going to be discussed further). Before starting to use the classifiers, we are going to extract features and normalize the data from the dataset Brazilian Malware. First, we read the CSV using pandas:

In [1]:
import pandas as pd
# dataset location
data_path = "./datasets/brazilian-malware.csv"
# read CSV dataset
data = pd.read_csv(data_path, keep_default_na=False)

Then, we list all the features of the dataset, categorizing them into numerical (real and integer numbers) and textual (libraries, system calls, compilers, etc). We also get the label and select the columns to be ignored. 

In [2]:
# numerical attributes
NUMERICAL_ATTRIBUTES = ['BaseOfCode', 'BaseOfData', 'Characteristics', 'DllCharacteristics', 
                      'Entropy', 'FileAlignment', 'ImageBase', 'Machine', 'Magic',
                      'NumberOfRvaAndSizes', 'NumberOfSections', 'NumberOfSymbols', 'PE_TYPE',
                      'PointerToSymbolTable', 'Size', 'SizeOfCode', 'SizeOfHeaders',
                      'SizeOfImage', 'SizeOfInitializedData', 'SizeOfOptionalHeader',
                      'SizeOfUninitializedData']

# textual attributes
TEXTUAL_ATTRIBUTES = ['Identify', 'ImportedDlls', 'ImportedSymbols']

# label used to classify
LABEL = 'Label'

# attributes that are not used
UNUSED_ATTRIBUTES = ['FirstSeenDate', 'SHA1', 'TimeDateStamp']

Then, we get the labels and remove unused attributes:

In [3]:
label = data[LABEL].values
# remove unused attributes and label
for a in UNUSED_ATTRIBUTES:
    del data[a]
del data[LABEL]

After that, we split the dataset in half (the first one is for the training set and the last, for the testing set):

In [4]:
# split data in half
def split_data(data):
    # get mid of data
    mid = int((len(data) + 1)/2)
    # split data into train and test
    train_data = data[:mid]
    test_data = data[mid:]
    # return train and test data
    return(train_data, test_data)

In [5]:
# data, _ = split_data(data)
# label, _ = split_data(label)
train_data, test_data = split_data(data)
train_label, test_label = split_data(label)

Here we created a method to extract features, given a train and test set and a feature extractor:

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# extract features from textual attributes
def textual_feature_extraction(train_data, test_data, extractor=TfidfVectorizer(max_features=100)):
    vectorizer = extractor
    # train vectorizer
    vectorizer.fit(train_data)
    # transform train and test data to features
    train_features = vectorizer.transform(train_data)
    test_features = vectorizer.transform(test_data)
    # return train and test features
    return(train_features, test_features)

Obtain numerical attributes:

In [7]:
train_features = train_data[NUMERICAL_ATTRIBUTES].values
test_features = test_data[NUMERICAL_ATTRIBUTES].values

In [8]:
train_features.shape, test_features.shape

((25091, 21), (25090, 21))

Obtain textual attributes and append to features array already initialized:

In [9]:
import numpy as np
# extract features from each textual attribute
for a in TEXTUAL_ATTRIBUTES:
    # extract features from current attribute
    train_texts, test_texts = textual_feature_extraction(train_data[a], test_data[a])
    train_features = np.concatenate((train_features, train_texts.toarray()), axis=1)
    test_features = np.concatenate((test_features, test_texts.toarray()), axis=1)

In [10]:
train_features.shape, test_features.shape

((25091, 321), (25090, 321))

Then, we created a normalization method, which normalizes both training and testing set, given a scaler:

In [11]:
from sklearn.preprocessing import MinMaxScaler
def normalization(train_features, test_features, scaler=MinMaxScaler()):
    # train minmax
    scaler.fit(train_features)
    # transform features
    train_features_norm = scaler.transform(train_features)
    test_features_norm = scaler.transform(test_features)
    # return normalized train and test features
    return(train_features_norm, test_features_norm)

In [12]:
train_features_norm, test_features_norm = normalization(train_features, test_features)

From now on, we have a training set and a testing set ready for classification.

In [13]:
train_features_norm.shape, test_features_norm.shape

((25091, 321), (25090, 321))

### Scikit-Learn

Scikit-Learn [Pedregosa et al. 2011] is one of the most complete machine learning libraries, with many algorithms already implemented for classification, preprocessing, feature extraction and normalization. Due to that, we already used this library in [feature extraction section](04_features.ipynb) (to use TF-IDF, min-max and standardization). From all the presented algorithms, this library implements [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [SVM](https://scikit-learn.org/stable/modules/svm.html) classifiers, and [K-Means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) clustering algorithms. Note that the majority of the algorithms implemented by Scikit-Learn are designed for data in batches, where only batches collected during a given time are used (this time is usually ignored, allowing a random access of the data and the assumption that the data has a stationary distribution, i.e., they do not change as time goes by), without concerning about new data to be collected (in this case, if there is new data batches, it would be necessary to train a new model from the scratch to learn again with these new samples).

As shown in every code for each classifier below, all of them works in a similar way: first you initialize them with their parameters (which are detailed in their documentation pages), train them using the training data and labels (through the method *fit*) and then create a prediction of the testing data (through the method *predict*). The clustering algorithms use the method *fit_predict*, which takes as input only the data that you want to cluster and already train the model, returning the label of each sample.

#### KNN

KNN trained with $K=3$.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
# initialize classifier
clf = KNeighborsClassifier(n_neighbors=3)
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(25090,) (25090,)


#### Random Forest

Random Forest trained using $n_estimators=10$, i.e., 10 decision trees.

In [15]:
from sklearn.ensemble import RandomForestClassifier
# initialize classifier
clf = RandomForestClassifier(n_estimators=10)
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(25090,) (25090,)


#### SVM

SVM trained using a linear kernel.

In [16]:
from sklearn.svm import SVC
# initialize classifier
clf = SVC(kernel="linear")
# train classifier
clf.fit(train_features_norm, train_label)
# predict test classes
test_pred = clf.predict(test_features_norm)
# print test pred and real labels shape
print(test_pred.shape, test_label.shape)

(25090,) (25090,)


#### K-Means

K-Means trained with $K=2$.

In [17]:
from sklearn.cluster import KMeans
# initialize kmeans
clustering = KMeans(n_clusters=2)
# fit kmeans (note that we do not need to use labels here)
# and predict train classes using the clusters created
train_pred = clustering.fit_predict(train_features_norm)
# print train pred and real labels shape
print(train_pred.shape, train_label.shape)

(25091,) (25091,)


#### DBScan

DBScan with $\epsilon=3$ and $min\_samples=2$.

In [18]:
from sklearn.cluster import DBSCAN
# initialize dbscan
clustering = DBSCAN(eps=3, min_samples=2)
# fit kmeans (note that we do not need to use labels here)
# and predict train classes using the clusters created
train_pred = clustering.fit_predict(train_features_norm)
# print train pred and real labels shape
print(train_pred.shape, train_label.shape)

(25091,) (25091,)


### Scikit-Multiflow

[Scikit-Multiflow](https://scikit-multiflow.github.io/) is a machine learning library that follows "scikits philosophy" and focus on data streams problems. Different from the batches format, data streams are data accessed continuously (and sequential), usually in real time (which allows them to be used in real world), i.e., at each time a new sample can be seen, directly influencing the model, given that the distribution usually is non-stationary and has changes as time goes by [Albert Bifet 2018]. Figure below compares data in batches and stream: when using batch, the training and testing steps are well defined and the dataset is finite, different from a stream, where the data can be part of the training and testing set as time goes by (depending of the concept drift and concept evolution), that it, theoretically infinite, once new data can arrive at any moment.

<img src="imgs/data.png" width="800" align="center">

From the classifiers presented in this section, Scikit-Multiflow implements some adaptations (involving data streams problems) of [KNN](https://scikit-multiflow.github.io/scikit-multiflow/_autosummary/skmultiflow.lazy.KNNAdwin.html) (which already includes an ADWIN drift detector) and [Random Forest](https://scikit-multiflow.github.io/scikit-multiflow/_autosummary/skmultiflow.meta.AdaptiveRandomForest.html) (which already includes an ADWIN drift detector too and it is possible to select any drift detector and even disable it). From the drift detectors cited, all of them are implemented: [DDM](https://scikit-multiflow.github.io/scikit-multiflow/_autosummary/skmultiflow.drift_detection.DDM.html), [EDDM](https://scikit-multiflow.github.io/scikit-multiflow/_autosummary/skmultiflow.drift_detection.EDDM.html) and [ADWIN](https://scikit-multiflow.github.io/scikit-multiflow/_autosummary/skmultiflow.drift_detection.ADWIN.html). At the moment of this work, none of the clustering algorithms were implemented by this library.

The main difference of Scikit-Multiflow from Scikit-Learn is that the training method is called *partial_fit*, which allows the model to be updated with new samples. Below we show examples for KNN and Random Forest. Besides that, we use the class *DataStream* to create a stream using the test data. Thus, for each data in the stream, the model predicts its class and updates the model with a new data (at this same time, as both classifiers has a built-in drift detector, it is already verified if there was a concept drift to correctly update the model - discarding the old data if needed and keeping the newer ones, since the last warning, as already explained). In this implementation, we used a list to save all the predictions made by the model in the stream. All the classifiers and detectors are documented in their page, as already mentioned.

#### KNN

KNNAdwin initialized with $n\_neighbors=3$ and partially fitted with train data and label.

In [43]:
from skmultiflow.lazy.knn_adwin import KNNAdwin
from skmultiflow.data import DataStream
# initialize classifier
clf = KNNAdwin(n_neighbors=3)
# fit classifier
clf.partial_fit(train_features_norm, train_label)

KNNAdwin(leaf_size=30, max_window_size=9223372036854775807, n_neighbors=3,
         nominal_attributes=None)

Then, we create a data stream using the test data and iterate over it, predicting the samples and updating the model if needed (built-in implementation for this last step).

In [ ]:
# create a stream with test features
stream = DataStream(test_features_norm, test_label)
# prepare stream for use
stream.prepare_for_use()
# create prediction array
test_pred = []
# iterate over stream
while stream.has_more_samples():
    # get next sample features and label from stream
    sample_features, sample_label = stream.next_sample(1)
    # predict sample
    sample_pred = clf.predict(sample_features)
    # add predicted labels to test_pred
    for l in sample_pred:
        test_pred.append(l)
    # update model with new sample
    clf.partial_fit(sample_features, sample_label)
# turn test_pred into numpy array
test_pred = np.array(test_pred)
print(test_pred.shape, test_label.shape)

#### Random Forest

AdaptiveRandomForest initialized with $n\_estimators=10$, with an ADWIN drift detector (as default).

In [31]:
from skmultiflow.meta import AdaptiveRandomForest
from skmultiflow.data import DataStream
# initialize classifier
clf = AdaptiveRandomForest(n_estimators=10, disable_weighted_vote=True) # disable_weighted_vote=False produces a bug
# fit classifier
clf.partial_fit(train_features_norm, train_label)

AdaptiveRandomForest(binary_split=False, disable_weighted_vote=True,
                     drift_detection_method=ADWIN(delta=0.001), grace_period=50,
                     lambda_value=6, leaf_prediction='nba',
                     max_byte_size=33554432, max_features=18,
                     memory_estimate_period=2000000, n_estimators=10,
                     nb_threshold=0, no_preprune=False, nominal_attributes=None,
                     performance_metric='acc', random_state=None,
                     remove_poor_atts=False, split_confidence=0.01,
                     split_criterion='info_gain', stop_mem_management=False,
                     tie_threshold=0.05,
                     warning_detection_method=ADWIN(delta=0.01))

Similar to before, we create a stream and iterate over it.

In [32]:
# create a stream with test features
stream = DataStream(test_features_norm, test_label)
# prepare stream for use
stream.prepare_for_use()
# create prediction array
test_pred = []
# iterate over stream
while stream.has_more_samples():
    # get next sample features and label from stream
    sample_features, sample_label = stream.next_sample(1)
    # predict sample
    sample_pred = clf.predict(sample_features)
    # add predicted labels to test_pred
    for l in sample_pred:
        test_pred.append(l)
    # update model with new sample
    clf.partial_fit(sample_features, sample_label)
# turn test_pred into numpy array
test_pred = np.array(test_pred)
print(test_pred.shape, test_label.shape)

(12545,) (12545,)


#### Drift Detectors

To use a drift detector with any classifier, it is possible to follow the codes below. Here we are going to use HoeffdingTree, the same decision tree used by AdaptiveRandomForest. First, we initialize the model, train it with the training data and label, and initialize a secondary classifier which is going to be used in case a drift is detected.

In [31]:
from skmultiflow.trees import HoeffdingTree
# initialize classifier
clf = HoeffdingTree()
# fit classifier
clf.partial_fit(train_features_norm, train_label)
# initialize classifier 2
clf2 = HoeffdingTree()

Then, we initialize a DDM drift detector.

In [32]:
from skmultiflow.drift_detection import EDDM, DDM
# initialize drift detector
drift = DDM()

After that, we create a data stream using the test data.

In [33]:
from skmultiflow.data import DataStream
from skmultiflow.core import clone
# create a stream with test features
stream = DataStream(test_features_norm, test_label)
# prepare stream for use
stream.prepare_for_use()
# create prediction array
test_pred = []
# counter
count = 0
# drift points
drifts = []
# iterate over stream
while stream.has_more_samples():
    # get next sample features and label from stream
    sample_features, sample_label = stream.next_sample(1)
    # increase counter
    count += 1
    # predict sample
    sample_pred = clf.predict(sample_features)
    # add predicted labels to test_pred
    for l in sample_pred:
        test_pred.append(l)
    # add element to drift detector
    for e in sample_label == sample_pred:
        drift.add_element(e)
    # detect if warning or drift
    if drift.detected_warning_zone():
        # update classifier 2
        clf2.partial_fit(sample_features, sample_label)
    if drift.detected_change():
        # save drift point to array
        drifts.append(count)
        # change classifiers
        clf = clone(clf2)
        # initialize classifier 2 again
        clf2 = HoeffdingTree()
        # reset drift detector
        drift.reset()
    # update model with new sample
    clf.partial_fit(sample_features, sample_label)
# turn test_pred into numpy array
test_pred = np.array(test_pred)

In [35]:
# print drift points
print("Detected drifts in points {}.".format((drifts)))
# print shape
print(test_pred.shape, test_label.shape)

Detected drifts in points [19936, 20938, 21126, 21183, 24425, 24493, 24536, 24729].
(25090,) (25090,)


### Keras

TODO

#### Multi-Layer Perceptron

TODO

In [42]:
import keras
from keras.models import Sequential
from keras.layers import Dense
# converts labels to a categorical one-hot-vector
train_label_onehot = keras.utils.to_categorical(train_label, num_classes=2)
test_label_onehot = keras.utils.to_categorical(test_label, num_classes=2)
# initialize sequential network
model = Sequential()
# add fully-connected hidden layer with 200 units
model.add(Dense(200, activation='relu', input_dim=train_features_norm.shape[1]))
# add fully-connected hidden layer with 100 units
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(2, activation='softmax'))
# compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# fit model with data
model.fit(train_features_norm, train_label_onehot, validation_split=0.33, epochs=10, batch_size=128)
# predict classes
test_pred = model.predict_classes(test_features_norm)
print(test_pred.shape, test_label.shape)

Train on 8405 samples, validate on 4141 samples
Epoch 1/10
8405/8405 [==============================] - 1s 106us/step - loss: 0.0313 - acc: 0.9861 - val_loss: 5.1686 - val_acc: 0.5576
Epoch 2/10
8405/8405 [==============================] - 0s 36us/step - loss: 4.0635e-05 - acc: 1.0000 - val_loss: 6.4903 - val_acc: 0.5576
Epoch 3/10
8405/8405 [==============================] - 0s 37us/step - loss: 1.4403e-06 - acc: 1.0000 - val_loss: 6.9533 - val_acc: 0.5576
Epoch 4/10
8405/8405 [==============================] - 0s 37us/step - loss: 1.8051e-07 - acc: 1.0000 - val_loss: 7.0748 - val_acc: 0.5576
Epoch 5/10
8405/8405 [==============================] - 0s 37us/step - loss: 1.2155e-07 - acc: 1.0000 - val_loss: 7.0901 - val_acc: 0.5576
Epoch 6/10
8405/8405 [==============================] - 0s 35us/step - loss: 1.1462e-07 - acc: 1.0000 - val_loss: 7.0918 - val_acc: 0.5576
Epoch 7/10
8405/8405 [==============================] - 0s 39us/step - loss: 1.1261e-07 - acc: 1.0000 - val_loss: 7.0919 

## References

[Albert Bifet 2018] Albert Bifet, Ricard Gavalda, G. H. B. P. (2018). Machine Learning for Data Streams with Practical Examples in MOA. MIT Press. https://moa.cms.waikato.ac.nz/book/.


[Almeida et al. 2015] Almeida, P., Oliveira, L., Britto, A., and Sabourin, R. (2015). Dealing with concept drifts using dynamic ensembles of classifiers. Tesis presented as partial requirement for the degree of Doctor. Graduate Program in Informatics, Sector of Exact Sciences, Universidade Federal do Paraná.

[Baena-Garcıa et al. 2006] ´ Baena-Garcıa, M., del Campo-Ávila, J., Fidalgo, R., Bifet, ´A., Gavaldà, R., and Morales-Bueno, R. (2006). Early drift detection method.

[Bifet and Gavaldà 2007] Bifet, A. and Gavaldà, R. (2007). Learning from timechanging data with adaptive windowing. volume 7.

[Bishop 2006] Bishop, C. M. (2006). Pattern Recognition and Machine Learning (Information Science and Statistics). Springer-Verlag, Berlin, Heidelberg.

[Breiman 2001] Breiman, L. (2001). Random forests. Mach. Learn., 45(1):5–32.

[Chollet et al. 2015] Chollet, F. et al. (2015). Keras. https://github.com/fchollet/keras.

[Gama et al. 2004] Gama, J., Medas, P., Castillo, G., and Rodrigues, P. (2004). Learning with drift detection. In Bazzan, A. L. C. and Labidi, S., editors, Advances in Artificial Intelligence – SBIA 2004, pages 286–295, Berlin, Heidelberg. Springer Berlin Heidelberg.

[Gama et al. 2014] Gama, J. a., Žliobaite, I., Bifet, A., Pechenizkiy, M., and Bouchachia, ˙A. (2014). A survey on concept drift adaptation. ACM Comput. Surv., 46(4):44:1–44:37.

[Haykin 2009] Haykin, S. (2009). Neural Networks and Learning Machines. Number v. 10 in Neural networks and learning machines. Prentice Hall.

[Manning et al. 2008a] Manning, C. D., Raghavan, P., and Schütze, H. (2008a). Introduction to Information Retrieval. Cambridge University Press, New York, NY, USA.

[Michie et al. 1994] Michie, D., Spiegelhalter, D. J., Taylor, C. C., and Campbell, J., editors (1994). Machine Learning, Neural and Statistical Classification. Ellis Horwood, Upper Saddle River, NJ, USA.

[Milgram et al. 2006] Milgram, J., Cheriet, M., and Sabourin, R. (2006). “One Against One” or “One Against All”: Which One is Better for Handwriting Recognition with SVMs? In Lorette, G., editor, Tenth International Workshop on Frontiers in Handwriting Recognition, La Baule (France). Université de Rennes 1, Suvisoft. http://www.suvisoft.com.

[Mitchell 1997] Mitchell, T. M. (1997). Machine Learning. McGraw-Hill, Inc., New York, NY, USA, 1 edition.

[Montiel et al. 2018] Montiel, J., Read, J., Bifet, A., and Abdessalem, T. (2018). Scikitmultiflow: A multi-output streaming framework. Journal of Machine Learning Research, 19(72):1–5.

[Pedregosa et al. 2011] Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., and Duchesnay, E. (2011). Scikitlearn: Machine learning in Python. Journal of Machine Learning Research, 12:2825–2830.

[Wang et al. 2011] Wang, S., Schlobach, S., and Klein, M. (2011). Concept drift and how to identify it. Web Semantics: Science, Services and Agents on the World Wide Web, 9(3):247 – 265. Semantic Web Dynamics Semantic Web Challenge, 2010.

---

[**<< Previous Section**](04_features.ipynb) | [**Next Section >>**](06_evaluation.ipynb)